In [1]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [104]:
# 念のためパスの確認
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


In [105]:
import MeCab
import os,json,random

In [106]:
# 夏目漱石のファイルを読み込んだ際の関数
# ファイルを読み込み、マルコフ辞書に登録する関数を作成する
def make_natume(filename):

  # 辞書に登録されていない場合処理を行う
  if not os.path.exists(dict_file_natume): 

    # withを使用してテキストファイルを開く
    with open(filename) as f:
        # 1行ずつ読み込む
        line = f.readline()

        # テキストファイルの最終行までループする
        while line:
            text = line.rstrip()

            # 形態素解析
            # 下記のパスは、環境設定時に表示された値にする
            tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
            tagger.parse('')
            node =  tagger.parseToNode(text)

            # 形態素解析の結果から、単語と品詞情報を抽出
            wordlist = []
            while node is not None:
                hinshi = node.feature.split(',')[0]
                if  hinshi not  in ['BOS/EOS']:
                    wordlist.append([node.surface,hinshi])
                node = node.next
            
            # マルコフ連鎖の辞書に登録
            regist_dic(wordlist)

            line = f.readline()

In [107]:
# 森鴎外のファイルを読み込んだ際の関数
# ファイルを読み込み、マルコフ辞書に登録する関数を作成する
def make_ougai(filename):

  # 辞書に登録されていない場合処理を行う
  if not os.path.exists(dict_file_ougai): 

    # withを使用してテキストファイルを開く
    with open(filename) as f:
        # 1行ずつ読み込む
        line = f.readline()

        # テキストファイルの最終行までループする
        while line:
            text = line.rstrip()

            # 形態素解析
            # 下記のパスは、環境設定時に表示された値にする
            tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
            tagger.parse('')
            node =  tagger.parseToNode(text)

            # 形態素解析の結果から、単語と品詞情報を抽出
            wordlist = []
            while node is not None:
                hinshi = node.feature.split(',')[0]
                if  hinshi not  in ['BOS/EOS']:
                    wordlist.append([node.surface,hinshi])
                node = node.next
            
            # マルコフ連鎖の辞書に登録
            regist_dic(wordlist)

            line = f.readline()

In [108]:
# 芥川龍之介のファイルを読み込んだ際の関数
# ファイルを読み込み、マルコフ辞書に登録する関数を作成する
def make_akutagawa(filename):

  # 辞書に登録されていない場合処理を行う
  if not os.path.exists(dict_file_akutagawa): 

    # withを使用してテキストファイルを開く
    with open(filename) as f:
        # 1行ずつ読み込む
        line = f.readline()

        # テキストファイルの最終行までループする
        while line:
            text = line.rstrip()

            # 形態素解析
            # 下記のパスは、環境設定時に表示された値にする
            tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
            tagger.parse('')
            node =  tagger.parseToNode(text)

            # 形態素解析の結果から、単語と品詞情報を抽出
            wordlist = []
            while node is not None:
                hinshi = node.feature.split(',')[0]
                if  hinshi not  in ['BOS/EOS']:
                    wordlist.append([node.surface,hinshi])
                node = node.next
            
            # マルコフ連鎖の辞書に登録
            
            regist_dic(wordlist)

            line = f.readline()

In [109]:
dict_file_natume = 'natume_dict.json'
dict_file_ougai = 'ougai_dict.json'
dict_file_akutagawa = 'akutagawa_dict.json'
dic = {}

# 辞書への登録
def regist_dic(wordlist):
    global dic
    w1 = ''
    w2 = ''
    
    # 要素が3未満の場合は、何もしない
    if len(wordlist) < 3 :
      return

    for w in wordlist :
        word = w[0]

        if word == '' or  word == '\r\n' or word == '\n' :
            continue

        # 辞書に単語を設定
        if w1 and w2 :
            set_dic(dic,w1, w2, word)

        # 文末を表す語の場合、連鎖をクリアする
        if word == '。' or word == '?' or  word == '？' :
            w1 = ''
            w2 = ''
            continue
        # 次の前後関係を登録するために、単語をスライド
        w1, w2 = w2, word
    
    # 辞書を保存
    if human == 1:
      json.dump(dic, open(dict_file_natume, 'w'))
    elif human == 2:
      json.dump(dic, open(dict_file_ougai, 'w'))
    elif human == 3:
      json.dump(dic, open(dict_file_akutagawa, 'w'))

# 辞書に単語を設定
def set_dic(dic, w1, w2, w3):
    # 新しい単語の場合は、新しい辞書オブジェクトを作成
    # ・dicのキーにw1が存在しないときは、キーを追加し、空の辞書をセットする
    if w1 not in dic : dic[w1] = {}

    # ・dic[w1]のキーにw2が存在しないときは、キーを追加し、空の辞書をセットする
    if w2 not in dic[w1] : dic[w1][w2] = {}

    # ・dic[w1][w2]のキーにw3が存在しないときは、キーを追加し、0をセットする
    if w3 not in dic[w1][w2] : dic[w1][w2][w3] = 0

    # 単語の出現数をインクリメントする
    dic[w1][w2][w3] += 1

In [110]:
# 応答文の作成
def make_response(word):
    res = []
    # 「名詞」/「形容詞」/「動詞」は、文章の意図を示していることが多いと想定し、始点の単語とする。
    w1 = word
    res.append(w1)
    w2 = word_choice(dic[w1])
    res.append(w2)
    while True:
        # w1,w2の組み合わせから予想される、単語を選択
        if w1 in dic and w2 in dic[w1] : w3 = word_choice(dic[w1][w2])
        else : w3 = ""
        res.append(w3)
        # 文末を表す語の場合、作文を終了
        if w3 == "。" or w3 == "?" or  w3 == "？"  or w3 == "" :  break
        # 次の単語を選択するために、単語をスライド
        w1, w2 = w2, w3
    return "".join(res)        
        
def word_choice(candidate):
    keys = candidate.keys()
    return random.choice(list(keys))

# メイン処理
def play_chat():

  # 辞書がすでに存在する場合は、最初に読み込む
  if human == 1:
    if os.path.exists(dict_file_natume):
            dic = json.load(open(dict_file_natume,"r"))
  elif human == 2:
    if os.path.exists(dict_file_ougai):
            dic = json.load(open(dict_file_ougai,"r"))
  elif human == 3:
    if os.path.exists(dict_file_akutagawa):
            dic = json.load(open(dict_file_akutagawa,"r"))
          
  while True:
      # 標準入力から入力を受け付け、「さようなら」が入力されるまで続ける
      text = input("You -> ")
      if text == "" or text == "また会いましょう" : 
          print("Bot -> また会いましょう")
          break

      # 文章整形
      if text[-1] != "。" and text[-1] != "?" and text[-1] != "？" : text +="。"
      
      # 形態素解析
      tagger = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
      tagger.parse("")
      node =  tagger.parseToNode(text)
      
      # 形態素解析の結果から、単語と品詞情報を抽出
      wordlist = []
      while node is not None:
          hinshi = node.feature.split(",")[0]
          if  hinshi not  in ["BOS/EOS"]:
              wordlist.append([node.surface,hinshi])
          node = node.next

      # 応答文の作成
      for w in wordlist:
          word = w[0]
          hinshi = w[1]
          # 品詞が「感動詞」の場合は、単語をそのまま返す
          if hinshi in [ "感動詞"] : 
              print("Bot -> " + word)
              break
          # 品詞が「名詞」「形容詞」「動詞」の場合で、かつ、辞書に単語が存在する場合は、作文して返す
          elif (hinshi in [ "名詞" ,"形容詞","動詞"]) and (word in dic):
              print("Bot -> " + make_response(word))
              break

In [119]:
# 正しい値が入力されるまでループする
while True:

  # 著者を選択してもらい数字を入力してもらう
  print('１：夏目漱石')
  print('２：森鴎外')
  print('３：芥川龍之介')
  try:
    human = int(input('著者を入力してください：'))
  except Exception as e:
    print(e)

  # make_markof関数を使用して、選んだ著者ののマルコフ辞書を作成する
  try:
    if human == 1:
      print('夏目漱石が選択されました')
      make_natume('natume.txt')
      break
    elif human == 2:
      print('森鴎外が選択されました')
      make_ougai('ougai.txt')
      break
    elif human == 3:
      print('芥川龍之介が選択されました')
      make_akutagawa('akutagawa.txt')
      break
    else:
      print('指定した番号を入力してください')
  except Exception as e:
    print(e)

# play_chat関数を呼び出す
play_chat()

１：夏目漱石
２：森鴎外
３：芥川龍之介
著者を入力してください：3
芥川龍之介が選択されました
You -> 私は魚が好きです
Bot -> 私はさほどの都会人種が住んでいるという意味がまるで出なかった事のできない人でなく、高き帽を被ってやって来た。
You -> エリスはどうした
Bot -> エリスは打笑みつゝこれを実行することゝなり、われと共に東にかへる心なきか、君が学問こそわが測り知る所ならね、語学のみにて、或日相沢は跡より来ても心持は同じ言葉を二、三名と共に、余を拉し去りて、理由もないと、剛気ある父の手蹟であって、上にもなかった。
You -> 
Bot -> また会いましょう
